In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [2]:
# Set parameters
img_size = 224  # Image dimensions for resizing
batch_size = 32
epochs = 10
data_dir = "X-Rays"  # Directory containing "Pneumonia", "COVID-19", "Normal" subfolders


In [3]:
# Data Augmentation and Data Loading
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 989 images belonging to 3 classes.
Found 245 images belonging to 3 classes.


In [4]:
# ZF-Net Model
def build_zf_net(input_shape=(img_size, img_size, 3)):
    model = Sequential([
        Conv2D(96, (7, 7), activation='relu', strides=(2, 2), input_shape=input_shape),
        MaxPooling2D((3, 3), strides=(2, 2)),
        Conv2D(256, (5, 5), activation='relu', strides=(2, 2)),
        MaxPooling2D((3, 3), strides=(2, 2)),
        Conv2D(384, (3, 3), activation='relu'),
        Conv2D(384, (3, 3), activation='relu'),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# VGG-19 Model
def build_vgg_19(input_shape=(img_size, img_size, 3)):
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        vgg19,
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# Initialize and Train ZF-Net
zf_net = build_zf_net()
history_zf_net = zf_net.fit(train_generator, epochs=epochs, validation_data=validation_generator)

c:\Users\anshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


c:\Users\anshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.3274 - loss: 1.1018 - val_accuracy: 0.3306 - val_loss: 1.0982
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.3388 - loss: 1.0987 - val_accuracy: 0.3306 - val_loss: 1.0982
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.3697 - loss: 1.0982 - val_accuracy: 0.3388 - val_loss: 1.0964
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.3828 - loss: 1.0936 - val_accuracy: 0.4408 - val_loss: 1.0884
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.4442 - loss: 1.0520 - val_accuracy: 0.4694 - val_loss: 1.0595
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.5244 - loss: 0.9491 - val_accuracy: 0.5347 - val_loss: 0.9635
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.5494 - loss: 0.8511 - val_accuracy: 0.5755 - val_loss: 0.9156
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.6369 - loss: 0.7849 - val_accuracy: 0.4816 - val_loss: 1.0031
Epo

In [10]:
# Initialize and Train VGG-19
vgg_19 = build_vgg_19()
history_vgg_19 = vgg_19.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
 1/31 ━━━━━━━━━━━━━━━━━━━━ 56:40 113s/step - accuracy: 0.5000 - loss: 1.2129


KeyboardInterrupt



In [9]:
# Plot accuracy
ax1.plot(history_zf_net.history['accuracy'], label='ZF-Net Train Accuracy')
ax1.plot(history_zf_net.history['val_accuracy'], label='ZF-Net Validation Accuracy')
ax1.plot(history_vgg_19.history['accuracy'], label='VGG-19 Train Accuracy')
ax1.plot(history_vgg_19.history['val_accuracy'], label='VGG-19 Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# Plot loss
ax2.plot(history_zf_net.history['loss'], label='ZF-Net Train Loss')
ax2.plot(history_zf_net.history['val_loss'], label='ZF-Net Validation Loss')
ax2.plot(history_vgg_19.history['loss'], label='VGG-19 Train Loss')
ax2.plot(history_vgg_19.history['val_loss'], label='VGG-19 Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.show()

NameError: name 'ax1' is not defined

In [ ]:
def evaluate_and_plot_confusion_matrix(model, generator, model_name):
    # Predict on validation data
    y_pred = np.argmax(model.predict(generator), axis=1)
    y_true = generator.classes
    labels = list(generator.class_indices.keys())

    # Plot Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()

# Display Confusion Matrices for Both Models
evaluate_and_plot_confusion_matrix(zf_net, validation_generator, "ZF-Net")
evaluate_and_plot_confusion_matrix(vgg_19, validation_generator, "VGG-19")